In [2]:
%load_ext autoreload
%autoreload 2
%config InlineBackend.figure_format='retina'

import os
import sys
import warnings
warnings.simplefilter('ignore')

from IPython.display import display

# Add modules paths to system path
module_paths = ['.']
for module_path in module_paths:
    if os.path.abspath(os.path.join(module_path)) not in sys.path:
        sys.path.append(module_path)
    
import pickle
import numpy as np
import pandas as pd
import builtins

import matplotlib.pyplot as plt

from ipywidgets import interact, fixed
import ipywidgets as widgets

from struct_opt_dms.utils import save, load
from struct_opt_dms.dataset import load_dataframe
from struct_opt_dms.interactive_analysis import initialize, prep_power, select_frequency_range

current_input_id = -1
# current_input_id = 29

input_filterbanks_repository = '../data/filterbanks/R3/'
state_variable_name = 'df_arts_R3'
reload_state = True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Prepare bursts metadata
(detection parameters, repository name, local and arts paths) 

In [3]:
df_R3 = load_dataframe(input_filterbanks_repository, state_variable_name, reload=reload_state)
# df = df_R3[df_R3['struct_opt_dm'].isnull()]

Reloading dataframe df_arts_R3


# Load input data

In [4]:
# Main
verbose = True

# Go from one file to the other in the list in ascending (True) or decending (False) order
# `None` will reload the same file.
incr = True

if incr:
    current_input_id += 1
elif incr is not None:
    current_input_id -= 1

print ('Current burst (index=%d):' % current_input_id)
print (df_R3.iloc[current_input_id])
print ()

if df_R3.iloc[current_input_id]['file_location'] == '':
    if incr:
        current_input_id += 1
    elif incr is not None:
        current_input_id -= 1
    
# Global variables for interaction in next panel
global struct_opt_dm, struct_opt_dm_err
struct_opt_dm = 0
struct_opt_dm_err = 0
spectra, dm_trials, filename = initialize(df_R3.iloc[current_input_id]['file_location'], 
                                          df_R3.iloc[current_input_id]['detection_dm'],
                                          df_R3.iloc[current_input_id]['detection_downsampling'],
#                                           around_peak=False,
                                          verbose=verbose)
detection_mjd = df_R3.iloc[current_input_id]['detection_mjd']
power_vs_dm, d_power_vs_dm = prep_power(spectra, dm_trials, verbose=verbose)


Current burst (index=0):
detection_folder                                     2020-05-11-14:40:00.R3
observation_datetimes                                 2020-05-11T14:40:00.0
detection_time                                                       1889.1
detection_mjd                                                       58980.6
detection_dm                                                         348.06
detection_downsampling                                                   25
detection_snr                                                         58.14
filename                                 CB00_10.0sec_dm0_t01889_sb-1_tab00
file_location             ../data/filterbanks/R3/CB00_10.0sec_dm0_t01889...
struct_opt_dm                                                       348.603
struct_opt_dm_err                                                  0.183365
detection_isot                                   2020-05-11 15:11:29.100000
detection_phase                                      0 days 06:

# Interactive visualisation

In [5]:
fig = plt.figure(constrained_layout=True, figsize=(10, 7))
gs = fig.add_gridspec(15, 6)

interact(
    select_frequency_range, 
    spectra = fixed(spectra), 
    dm_trials = fixed(dm_trials),
    power_vs_dm = fixed(power_vs_dm),
    d_power_vs_dm  = fixed(d_power_vs_dm),
    fig=fixed(fig),
    gs=fixed(gs),
    fluct_id_low = widgets.IntSlider(min = 0, 
                                     max = power_vs_dm.shape[0], 
                                     step = 1, 
                                     value = 0, 
                                     continuous_update=False), 
    fluct_id_high = widgets.IntSlider(min = 0, 
                                      max = power_vs_dm.shape[0], 
                                      step = 1, 
                                      value = power_vs_dm.shape[0], 
                                      continuous_update=False), 
    freq_id_low = widgets.IntSlider(min = 0, 
                                    max = spectra.data.shape[0], 
                                    step = 1, 
                                    value = 0, 
                                    continuous_update=False),
    freq_id_high = widgets.IntSlider(min = 1, 
                                     max = spectra.data.shape[0], 
                                     step = 1, 
                                     value = spectra.data.shape[0], 
                                     continuous_update=False),
    t0 = widgets.IntSlider(min = 0, 
                           max = spectra.data.shape[1], 
                           step = 1, 
                           value = 0, 
                           continuous_update=False), 
    t1 = widgets.IntSlider(min = 1, 
                           max = spectra.data.shape[1], 
                           step = 1, 
                           value = spectra.data.shape[1], 
                           continuous_update=False),
    ds_freq = widgets.IntSlider(min=1, 
                                max=32, 
                                step=1, 
                                value=1, 
                                continuous_update=False),
    ds_time = widgets.IntSlider(min=1, 
                                max=32, 
                                step=1, 
                                value=1, 
                                continuous_update=False),
    delta_dm = widgets.FloatSlider(min = -10, 
                                   max = 10, 
                                   step = 0.01, 
                                   value = 0, 
                                   continuous_update=False),
    smooth = widgets.IntSlider(min=0, 
                               max=4, 
                               step=1, 
                               value=0, 
                               continuous_update=False),
    
)

# Save button
button = widgets.Button(description="Save figure")
display(button)

button_raw = widgets.Button(description="Save raw figure")
display(button_raw)

def save_to_df():
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'struct_opt_dm'] = builtins.struct_opt_dm 
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'struct_opt_dm_err'] = builtins.struct_opt_dm_err
    
    # Widgets values
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'fluct_id_low'] = builtins.fluct_id_low
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'fluct_id_high'] = builtins.fluct_id_high
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'freq_id_low'] = builtins.freq_id_low
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'freq_id_high'] = builtins.freq_id_high
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 't0'] = builtins.t0
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 't1'] = builtins.t1
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'ds_freq'] = builtins.ds_freq
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'ds_time'] = builtins.ds_time
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'delta_dm'] = builtins.delta_dm
    df_R3.loc[df_R3['detection_mjd'] == detection_mjd, 'smooth'] = builtins.smooth
    
    save(state_variable_name, df_R3)
    df_R3.to_csv('arts_r3.csv', index=False)

def check_dir(folder):
    if not os.path.exists(folder):
        os.makedirs(folder)
    
def save_figure(b):
    check_dir('images/manual_opt')
    check_dir('images/manual_opt/data')
    
    print ("saved to images/manual_opt/%s.png" % (filename.split('/')[-1].split('.fil')[0]))
    fig.savefig("images/manual_opt/%s.png" % (filename.split('/')[-1].split('.fil')[0]), dpi=300)

    with open("images/manual_opt/data/%s_waterfall.npy" % (filename.split('/')[-1].split('.fil')[0]), 'wb') as f:
        np.savetxt(f, builtins.sub_waterfall)
    with open("images/manual_opt/data/%s_fluctuation.npy" % (filename.split('/')[-1].split('.fil')[0]), 'wb') as f:
        np.savetxt(f, builtins.power_vs_dm)
        
    save_to_df()
        
def save_figure_raw(b):
    check_dir('images/manual_opt/raw')
    check_dir('images/manual_opt/raw/data')
    
    print ("saved to images/manual_opt/raw/%s.png" % (filename.split('/')[-1].split('.fil')[0]))
    fig.savefig("images/manual_opt/raw/%s.png" % (filename.split('/')[-1].split('.fil')[0]), dpi=300)

    with open("images/manual_opt/raw/data/%s_waterfall.npy" % (filename.split('/')[-1].split('.fil')[0]), 'wb') as f:
        np.savetxt(f, builtins.sub_waterfall)
    with open("images/manual_opt/raw/data/%s_fluctuation.npy" % (filename.split('/')[-1].split('.fil')[0]), 'wb') as f:
        np.savetxt(f, builtins.power_vs_dm)
        
    save_to_df()

button.on_click(save_figure)
button_raw.on_click(save_figure_raw)


<Figure size 720x504 with 0 Axes>

interactive(children=(IntSlider(value=0, continuous_update=False, description='fluct_id_low', max=610), IntSli…

Button(description='Save figure', style=ButtonStyle())

Button(description='Save raw figure', style=ButtonStyle())

# Plot DM vs Phase / time

In [ ]:
if 'detection_phase' not in df_R3.keys():
    phase0 = 58369.30
    t0 = Time(phase0, format='mjd')
    period = 16.28
    df_R3['detection_phase'] = df_R3['detection_mjd'].apply(lambda x: ((x - t0.mjd) % period) / period)
    

# df = df_R3.loc[(df_R3['detection_mjd'] > 58960) & (df_R3['detection_mjd'] < 58985)]
df = df_R3

plt.scatter(df['detection_phase'], df['struct_opt_dm'])
plt.errorbar(df['detection_phase'], df['struct_opt_dm'], yerr=df['struct_opt_dm_err'], ls='none')
plt.hlines(np.nanmedian(df['struct_opt_dm']), 
           np.min(df['detection_phase']), 
           np.max(df['detection_phase']), 
           linestyle='dashed', 
           colors='black', 
           alpha = 0.5)
plt.annotate(r'$\~{\tt{DM}}$ = %.2f pc/cc' % (np.nanmedian(df['struct_opt_dm'])), 
            xy=(0, 0.5),
            xycoords='axes fraction',
            xytext=(0.05, 0.85),)
plt.xlabel('Phase')
plt.ylabel('DM (pc/cc)')

plt.tight_layout()
plt.savefig('images/DM_v_Time_R3.png', dpi=300)

df_R3[['detection_isot', 
       'detection_mjd', 
       'detection_phase', 
       'struct_opt_dm', 
       'struct_opt_dm_err']].to_csv('R3_arts.csv', index=False)

# Dev Zone

In [ ]:
df_R3.iloc[1]

In [ ]:
current_input_id

In [7]:
df_R3[['detection_mjd', 'struct_opt_dm']].to_csv("filename.csv", )

,detection_mjd,struct_opt_dm
0,58980.632976,348.603252
1,58980.528927,348.699549
2,58997.241802,348.239065
3,58981.384389,350.680018
4,58997.272650,NaN
5,58980.388901,349.443759
6,58997.360973,348.871867
7,58997.157893,348.871654
8,58980.661888,348.871761
9,58996.347967,346.747169
